# Course 2 : Tokenization

The slides of the course are available [here](https://github.com/NathanGodey/AdvancedNLP/raw/main/slides/pdf/course2_tokenization.pdf)

## Part 0 : Data preparation

In this part, we'll install some libraries and prepare text datasets.

In [ ]:
!pip install datasets > /dev/null

#### Question 1
Using the documentation of the [`load_dataset`](https://huggingface.co/docs/datasets/v2.12.0/en/package_reference/loading_methods#datasets.load_dataset) function, download a <ins>natural language</ins> dataset from [HuggingFace's Hub](https://huggingface.co/datasets?sort=downloads). When picking your dataset, think about the following questions:
- What language do I want to use?
- What kind of language style is best suited for general tokenization?
- How big should my dataset be?

In [ ]:
from datasets import load_dataset

my_dataset = load_dataset(...)

Generating train split: 0 examples [00:00, ? examples/s]

#### Question 2

Define a `pretokenize` function that splits an input string into word units. Try to deal with as many punctuation signs as possible. Add a `"_"` after every word in the style of BPE.

**Example**
```python
pretokenize("My cat is blue!") -> ["My_", "cat_", "is_", "blue", "!"]
```


In [ ]:
def pretokenize(input_string):
  ...

pretokenize("My cat is blue!")

#### Question 3
Using the [`map`](https://huggingface.co/docs/datasets/process#map) method, generate a pretokenized version of your dataset.

In [ ]:
pretokenized_ds = ...

## Part 1 : Byte-Pair Encoding (BPE)

Let's get the size of your corpus down to a testable one:

In [ ]:
small_pretokenized_ds = pretokenized_ds.select(range(100))

#### Question 4 - Inference
Define a function that computes a BPE tokenization given a list of possible merges `merges` and a string `input_string`. It should return a list of string tokens.

In [ ]:
from typing import List

def bpe_inference(input_string, merges):
  ...

In [ ]:
test_input = list("education")
test_vocab = [("e", "d"), ("u", "c"), ("a", "t"), ("o", "n"), ("at", "i"), ("ed", "uc"), ("at", "ion")]

bpe_inference(input_string=test_input, vocabulary=test_vocab)

#### Question 5

Turn every word in your dataset into a list of characters.

In [ ]:
def char_split(sample_string):
  ...

In [ ]:
small_pretokenized_ds = small_pretokenized_ds.map(...)

#### Question 5

Extract a starting `vocabulary` from the pretokenized dataset. It will be a dictionary mapping every character encountered in `small_pretokenized_ds` to its number of occurences.

In [ ]:
def get_init_vocab(dataset):
  ...

In [ ]:
get_init_vocab(small_pretokenized_ds)

#### Question 6
Given a dataset `dataset` of list of tokenized sentences, create a function that finds the most common pair of consecutive tokens

In [ ]:
def get_most_common_pair(dataset):
  ...

In [ ]:
test_vocab = {
    "a": 16,
    "b": 293,
    "c": 318
}
get_most_common_pair(small_pretokenized_ds, test_vocab)

#### Question 7
We now need to create a function that:
- adds the most common pair `merged_pair` into the `merges` object
- merges the `merged_pair` occurences in the dataset
- updates the `vocabulary` counts accordingly

In [ ]:
def merge(merged_pair, merges, vocabulary, dataset):
  # Add merged_pair to merges

  # Find occurences corresponding to merged_pair in dataset and merge them together (+ keep count!)

  # Update the vocabulary counts: add the merged_pair entry, and change the counts of the elements of the pair

  return merges, vocabulary, dataset

In [ ]:
test_merged_pair = ("aa", "b")
test_merges = [("a", "a"), ("a", "b")]
test_vocabulary = {"a": 2, "b": 4, "aa": 3, "ab": 1}
test_dataset = [
    ["ab", "b", "b", "aa", "b", "a"],
    ["aa", "b", "aa", "ab", "a"]
]

merge(test_merged_pair, test_merges, test_vocabulary, test_dataset)

#### Question 8


Finally, we need to define a `cleanup` function, that removes unused items from the vocabulary:

In [ ]:
def cleanup(vocabulary):
  ...

In [ ]:
cleanup({"a": 4, "b": 5, "if you see me there's a bug": 0})

#### Question 9
Let's now reset out small dataset to pack the whole BPE step in a function:

In [ ]:
small_pretokenized_ds = pretokenized_ds.select(range(100))

Everything is now setup for us to apply the BPE algorithm. Put the previous functions in the right order and iterate 100 steps of the BPE algorithm on any given dataset.

In [ ]:
from tqdm.notebook import tqdm

def bpe_train(dataset, num_iters=100):

  # Prepare dataset and get initial vocabulary

  # Don't forget to initialize the list of possible merges

  for num_iter in tqdm(range(num_iters)):
    # Get the most common pair in the dataset

    # Merge the most common pair and update all relevant objects

    # Cleanup the vocabulary

  # Return vocabulary and merges

#### Question 10

Using the previous `bpe_inference` function, try out a few sentences and see how well they are tokenized.

#### Question 11

Let's scale! We'll first increase the number of iterations. Modify the `bpe_train` function to get a track of the vocabulary size over iterations. After how many iterations is the vocabulary not getting bigger?

In [ ]:
def bpe_train(dataset, num_iters=100000):
  ...

In [ ]:
import matplotlib.pyplot as plt

vocab, merges, vocab_size_over_iterations = bpe_train(small_pretokenized_ds)

plt.plot(vocab_size_over_iterations)
plt.xlabel("iterations")
plt.ylabel("vocabulary size")
plt.show()

#### Question 12

To improve the behaviour of our BPE tokenizer, we'll now scale in terms of data. Let's train a BPE tokenizer on a bigger slice of your original dataset. Try to estimate the computation time properly, and to pick an appropriate size.

In [ ]:
training_pretokenized_ds = pretokenized_ds.select(range(...))

vocab, merges, vocab_size_over_iterations = bpe_train(training_pretokenized_ds, num_iters=...)

As in **Question 10**, use the tokenizer on a few of your own sentences, and see how it performs.

## Part 2 : WordPiece

#### Question 13
Adapt the above part for the WordPiece algorithm. Try to keep memory and time complexity as low as possible!

#### Question 14

How do WordPiece and BPE compare? You can compare training speed, quality of tokenization, vocabulary size...

## Part 3 : Unigram (open)


#### Question 15

Implement the Unigram tokenizer and train it